In [ ]:
%matplotlib inline
import fceulib
from fceulib import VectorBytes
import numpy as np
import time
import matplotlib.pyplot as plt
from fceu_help import *
from math import *
import pickle


In [ ]:
import cv2 

inputVec = fceulib.readInputs('videos/happylee4-smb-warpless.fm2')
emu = fceulib.runGame('mario.nes')
image_prefix = 'images/mario'

#inputVec = fceulib.readInputs('videos/Contra by 3D.fm2')
#emu = fceulib.runGame('contra.nes')

peekevery = 1


display = False
img_buffer = VectorBytes()
prev = VectorBytes()


start = 270
end = float('inf')

nametables = {}
nametables2 = {}
xScrolls = {}
motion = {}

for timestep, inp in enumerate(inputVec):
    emu.stepFull(inp,0x0)
    if not (timestep % peekevery == 0) or timestep < start  :
        continue
    if timestep > end:
        break
    if timestep % 100 == 0:
        print (100*float(timestep)/float(len(inputVec)))
    if xScrolls is not None:
        xScrolls[timestep]  = emu.xScroll
        
    if nametables is not None:
        nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
        attr = nt[960:1024]
        nt = nt[:960]
        nt = nt.reshape(30,32)
        attr = attr.reshape(8,8)
        full_attr = np.zeros([16,16])
        for xx in range(8):
            for yy in range(8):
                val = attr[yy,xx]
                tr = (val & 0x0c) >> 2
                tl = val & 0x03
                br = (val & 0xc0 ) >> 6
                bl = (val & 0x30 ) >> 4
                full_attr[yy*2,xx*2] = tl
                full_attr[yy*2,xx*2+1] = tr
                full_attr[yy*2+1,xx*2] = bl
                full_attr[yy*2+1,xx*2+1] = br
        full_attr = full_attr[:15,:]
        full_attr = np.kron(full_attr, np.ones((2,2)))
        nametables[timestep] = (nt,full_attr)
        
    if nametables2 is not None:
        nt = pointer_to_numpy(emu.fc.ppu.NTARAM)
        attr = nt[1024+960:2048]
        nt = nt[:960]
        nt = nt.reshape(30,32)
        attr = attr.reshape(8,8)
        full_attr = np.zeros([16,16])
        for xx in range(8):
            for yy in range(8):
                val = attr[yy,xx]
                tr = (val & 0x0c) >> 2
                tl = val & 0x03
                br = (val & 0xc0 ) >> 6
                bl = (val & 0x30 ) >> 4
                full_attr[yy*2,xx*2] = tl
                full_attr[yy*2,xx*2+1] = tr
                full_attr[yy*2+1,xx*2] = bl
                full_attr[yy*2+1,xx*2+1] = br
        full_attr = full_attr[:15,:]
        full_attr = np.kron(full_attr, np.ones((2,2)))
        nametables2[timestep] = (nt,full_attr)
        
    emu.imageInto(img_buffer)
    if timestep > start:
        screen = pointer_to_numpy(img_buffer)
        prev_screen = pointer_to_numpy(prev)
        
        screen =  cv2.cvtColor(screen.reshape([256,256,4])[:240,:,:3],cv2.COLOR_BGR2GRAY)
        prev_screen = cv2.cvtColor(prev_screen.reshape([256,256,4])[:240,:,:3],cv2.COLOR_BGR2GRAY)
        
        flow = cv2.calcOpticalFlowFarneback(prev_screen,screen, 0.5, 3, 15, 6, 5, 1.2, 0)
        notzero = (np.abs(flow[:,:,0]) + np.abs(flow[:,:,1])) > 1
        motion_percent = float(np.sum(notzero))/float(256*240)
        if motion_percent < 0.1:
            motion[timestep] = (0,0)
        else:
            flow = flow[notzero]
            val = np.ndarray.astype(np.round(flow),dtype='int32')
            highest = -1
            best = None
            collection = {}
            for ii in range(len(val)):
                t = tuple(val[ii,:])
                if t not in collection:
                    collection[t] = 0
                collection[t] += 1
                if collection[t] > highest:
                    highest = collection[t]
                    best = t
            
            motion[timestep] = best
            
    temp = prev
    prev = img_buffer
    img_buffer = temp
    
    if display:
        outputImage(emu, '{}_{}'.format(image_prefix,timestep),img_buffer)
        

pickle.dump({'nametables'nametables:,
             'nametables2':nametables2,
             'xScrolls':xScrolls,
             'motion':motion},open('background.pkl','wb'))


In [ ]:
#skip this if you just ran the previous one
temp = pickle.load(open('background.pkl'))
nametables = temp['nametables']
nametables2= temp['nametables']
xScrolls = temp['xScrolls']
motion = temp['motion']



In [ ]:
#TODO -- figure out initial nametable position, keep track of nametable images throughout time using the motion vectors

